## Sveska za importovanje i pravljenje jobova za CAS-VIT mrežu

CAS-ViT se zasniva na Visual Transformer arhitekturi, više informacija na linku.

https://paperswithcode.com/paper/cas-vit-convolutional-additive-self-attention

(samo sam sortirao po broju parametara i uzeo ovu random mrezu, nemam pojma da li je dobra, i rad za sada ne razumem)

Ova sveska uz prateće fajlove automatski skida weightove za CAS-ViT model zadate veličine, i posle taj model šalje na AIHUB za testiranje.

Za sada sam pokrenuo samo xs model, koji se izvršava super brzo i ima manje parametara od MobileNetV2, doduše mnogo više layera koji su raspoređeni na više COMPUTE unita, što je koliko ja razumem dobro.

Ne znam da li je dobro da modele čuvamo na GIT-u (Drakula neka kaže kako bi to trebalo), pa sam sve fajlove koje se generišu stavio u gitignore.


In [1]:
import gdown
import torch
import rcvit
import torchviz
import torchsummary
import numpy as np
import qai_hub as hub
import requests
import input_getter
import make_jobs

c:\Users\Pro\Documents\___FAKULTET___\LPCV\LPCV_2025_T1\LPCV_2025_T1\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Pro\Documents\___FAKULTET___\LPCV\LPCV_2025_T1\LPCV_2025_T1\venv\Lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
c:\Users\Pro\Documents\___FAKULTET___\LPCV\LPCV_2025_T1\LPCV_2025_T1\venv\Lib\site-packages\timm\models\registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)


In [2]:
import os

# Model selection (options: "xs", "s", "m", "t")

modelID = "xs"

driveFileIDs = dict()

driveFileIDs["xs"] = "16wKcwF6QMW5w_lyPYnDKjMNuoxQDfrLK"
driveFileIDs["s"]  = "1facFRq8s8oelYUtK1fj3fcfdoWoKDBQQ"
driveFileIDs["m"]  = "13sQpSEf0h_uuh0jRy9V0yIW6ZsbDpVGy"
driveFileIDs["t"]  = "1NqoIUPbwBC91RTjTUvubAbOfGqo1VYT0"

networks = dict()
networks["xs"] = rcvit.rcvit_xs
networks["s"]  = rcvit.rcvit_s
networks["m"]  = rcvit.rcvit_m
networks["t"]  = rcvit.rcvit_t

file_id = driveFileIDs[modelID]

modelName = "CASVIT_{}".format(modelID)

file_path = modelName + ".pth"

if not os.path.exists(file_path):
    gdown.download(f"https://drive.google.com/uc?id={file_id}", file_path, quiet=False)
    print("Model downloaded successfully.")
else:
    print(f"File {file_path} already exists.")

File CASVIT_xs.pth already exists.


In [3]:
net: rcvit.RCViT = networks[modelID]()
print(net)

RCViT(
  (patch_embed): Sequential(
    (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(24, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
  )
  (network): ModuleList(
    (0): Sequential(
      (0): AdditiveBlock(
        (local_perception): LocalIntegration(
          (network): Sequential(
            (0): Conv2d(48, 48, kernel_size=(1, 1), stride=(1, 1))
            (1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48)
            (3): GELU(approximate='none')
            (4): Conv2d(48, 48, kernel_size=(1, 1), stride=(1, 1))
          )
        )
        (norm1): BatchNorm2d(48, eps=1e-05, momentum=0.1, aff

In [4]:
torchsummary.summary(net, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 24, 112, 112]             672
       BatchNorm2d-2         [-1, 24, 112, 112]              48
              ReLU-3         [-1, 24, 112, 112]               0
            Conv2d-4           [-1, 48, 56, 56]          10,416
       BatchNorm2d-5           [-1, 48, 56, 56]              96
              ReLU-6           [-1, 48, 56, 56]               0
            Conv2d-7           [-1, 48, 56, 56]           2,352
       BatchNorm2d-8           [-1, 48, 56, 56]              96
            Conv2d-9           [-1, 48, 56, 56]             480
             GELU-10           [-1, 48, 56, 56]               0
           Conv2d-11           [-1, 48, 56, 56]           2,352
 LocalIntegration-12           [-1, 48, 56, 56]               0
      BatchNorm2d-13           [-1, 48, 56, 56]              96
           Conv2d-14          [-1, 144,

In [5]:
# Loading pre-trained weights

weightDict = torch.load(file_path, map_location=torch.device("cpu"))
net.load_state_dict(weightDict["model"])

C:\Users\Pro\AppData\Local\Temp\ipykernel_26500\588262210.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  weightDict = torch.load(file_path, map_location=torch.device("c

<All keys matched successfully>

In [6]:
net.eval()
input = input_getter.random_input_getter()

net(input.get_input()).shape

torch.Size([1, 1000])

In [7]:
# Kada se uradi ovi jobovi na linkovima se moze naci vizuelizacija mreze i informacije za dalju analizu

c, p, i = make_jobs.compile_profile_inference(net, input)


Uploading tmpypvgrlgm.pt


100%|██████████| 13.3M/13.3M [00:02<00:00, 4.66MB/s]


Scheduled compile job (jg900w7mg) successfully. To see the status and results:
    https://app.aihub.qualcomm.com/jobs/jg900w7mg/

Waiting for compile job (jg900w7mg) completion. Type Ctrl+C to stop waiting at any time.
    ✅ SUCCESS                          
Scheduled profile job (jgdnnoy65) successfully. To see the status and results:
    https://app.aihub.qualcomm.com/jobs/jgdnnoy65/



Uploading dataset: 544kB [00:01, 349kB/s]                            1.78MB/s]


Scheduled inference job (j5722x1np) successfully. To see the status and results:
    https://app.aihub.qualcomm.com/jobs/j5722x1np/



In [8]:
input2 = input_getter.mug_image_getter()



if_job = make_jobs.inference_job(c.get_target_model(), input2.get_input())

Uploading dataset: 154kB [00:00, 239kB/s]                    <?, ?B/s]


Scheduled inference job (jp4nnv62g) successfully. To see the status and results:
    https://app.aihub.qualcomm.com/jobs/jp4nnv62g/



In [9]:
from make_jobs import inference_job_probabilities

inference_job_probabilities(if_job)

Waiting for inference job (jp4nnv62g) completion. Type Ctrl+C to stop waiting at any time.
    ✅ SUCCESS                          


tmp1d967s6h.h5: 100%|██████████| 14.5k/14.5k [00:00<00:00, 1.49MB/s]


Top-5 On-Device predictions:
967 b'espresso'           71.1%
504 b'coffee mug'         13.2%
968 b'cup'                13.2%
849 b'teapot'              0.5%
505 b'coffeepot'           0.4%
